Import neccessary libraries

In [ ]:
#Install the SDK
!pip3 install kfp --quiet
!python -m pip install --user --upgrade pip --quiet
!pip3 install pandas==0.23.4 matplotlib==3.0.3 scipy==1.2.1 scikit-learn==0.22 tensorflow==2.0 keras==1.2.2 --user --quiet

In [ ]:
import kfp
import kfp.components as comp

In [ ]:
def train():
    import pandas as pd
    import numpy as np
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    out_dir = "https://storage.googleapis.com/micro-access-290111/Hotel_Pipeline"
    
    df1 = pd.read_csv("https://storage.googleapis.com/micro-access-290111/Hotel_Review_Dataset(1).csv")
    df2 = pd.read_csv("https://storage.googleapis.com/micro-access-290111/Hotel_Review_Dataset(2).csv")
    df3 = pd.read_csv("https://storage.googleapis.com/micro-access-290111/Hotel_Review_Dataset(3).csv")
    
    df = pd.concat([df1,df2,df3])
    
    class Review_recommender():
        def __init__(self,data=df):
            self.data = data
            self.clean_df = self.data.groupby('Hotel_Name').agg({'Negative_Review':', '.join,'Positive_Review':', '.join}).reset_index()
            self.clean_df['Combined_Review'] = self.clean_df['Positive_Review'].astype(str) + self.clean_df['Negative_Review'].astype(str)
            self.clean_df["Combined_Review"] = self.clean_df["Combined_Review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))
            self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
            stop = stopwords.words('english')
            self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
            tfv = TfidfVectorizer(min_df=3, max_features =None,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3))
            self.tfv_matrix = tfv.fit_transform(self.clean_df['Combined_Review'])
            self.clean_df.drop(['Combined_Review','Positive_Review','Negative_Review'],1,inplace=True)
            del tfv,self.data,stop
            self.cos_sim = linear_kernel(self.tfv_matrix, self.tfv_matrix)
            self.indices = pd.Series(self.clean_df.index, index=self.clean_df['Hotel_Name']).drop_duplicates()
        
        def recommend(self,index):
            index = self.indices[index]
            sim_scores = list(enumerate(self.cos_sim[index]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
            hotel_indices = [i[0] for i in sim_scores]
            rec_list = []
            for i in self.clean_df['Hotel_Name'].iloc[hotel_indices]:
                rec_list.append(i)
    
    name = "Hotel Arena"
    recommender = Review_recommender()
    rec = recommender.recommend(name)
    rec_list = rec.rec_list
        
        
        
    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
        'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
        }]
    }

    # Exports two sample metrics:
    metrics = {
    'metrics': [{
            'name': 'Recommended Hotels',
            'list':  list(rec_list),
        },
        ]}

    from collections import namedtuple
    recommender_output = namedtuple('RecommenderOutput', ['rec_list', 'mlpipeline_ui_metadata', 'mlpipeline_metrics'])
    return recommender_output(rec_list, json.dumps(metadata), json.dumps(metrics))

Convert Python function into a pipeline operation

In [ ]:
train_op = comp.func_to_container_op(train, base_image='tensorflow/tensorflow:1.11.0-py3')

Test running the python function directly

Define the pipeline

Pipeline function has to be decorated with the @dsl.pipeline decorator

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Recommender pipeline',
   description='A toy pipeline that performs recommendation tasks.'
)
def rec_pipeline():
    #Passing pipeline parameter and a constant value as operation arguments
    train_task = train_op() #Returns a dsl.ContainerOp class instance. 

Submit the pipeline for execution

In [13]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}
experiment_name = 'experiment_1'

#Submit a pipeline run
kfp.Client(host='49b5cb790f037a10-dot-us-central2.pipelines.googleusercontent.com').create_run_from_pipeline_func(rec_pipeline, arguments=arguments)
# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(calc_pipeline, '{}.zip'.format(experiment_name))